### TorchIP: Descriptor
An example notebook that shows how to define and calulate descriptors for structures. 

In [ ]:
import sys
sys.path.append('../')

import torchip
from torchip.datasets import RunnerStructureDataset
from torchip.descriptors import AtomicSymmetryFunction  
from torchip.potentials import NeuralNetworkPotential
from torchip.utils import gradient, get_value
from torchip.structure import ElementMap
# from torchip.descriptors import CutoffFunction, G2, G3

import torch
import numpy as np
from pathlib import Path
from matplotlib.pylab import plt
from collections import defaultdict

In [ ]:
# torchip.set_logging_level(logging.DEBUG)
torchip.manual_seed(2020)
torchip.device.DEVICE = torch.device("cpu")
# torchip.dtype.FLOAT = torch.float64

In [ ]:
# Read structure
base_dir = './LJ' #Path('/home/hossein/n2p2/examples/nnp-train/H2O_RPBE-D3')
structures = RunnerStructureDataset(Path(base_dir, "input.data")) 
structures

In [ ]:
structures[0]

In [ ]:
# Potential
nnp = NeuralNetworkPotential(Path(base_dir, "input.nn"))
print("potential cutoff radius:", nnp.r_cutoff)
nnp

In [ ]:
# nnp.descriptor['O'].n_descriptor

In [ ]:
# nnp.fit_scaler(structures)
# !cat LJ/scaling.010.data

In [ ]:
dsc_tip = defaultdict(list)
for structure in structures:
    structure.set_cutoff_radius(nnp.r_cutoff)
    for element in nnp.elements:
        #print("Element:", element)
        dsc = nnp.descriptor[element](structure)
        # print(get_value(dsc).shape)
        dsc_tip[element].append(get_value(dsc))
        # scaled_dsc = nnp.scaler[element](dsc)
        # print(get_value(scaled_dsc))
    # break

dsc_tip = {elem: np.array(dsc_tip[elem]) for elem in dsc_tip}
for elem in dsc_tip:
    print(elem, dsc_tip[elem].shape)
    # print(dsc_tip[elem][0])

In [ ]:
dsc_n2p2 = defaultdict(list)
with open(Path(base_dir, 'function.data'), 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        n = int(line.rstrip("/n"))
        dsc_per_struct = defaultdict(list)
        for _ in range(n):
            line = f.readline().rstrip("/n").split()
            elem = ElementMap.get_element(int(line[0]))
            dsc_per_struct[elem].append([float(v) for v in line[1:]])
        f.readline()
        [dsc_n2p2[elem].append(dsc_per_struct[elem]) for elem in dsc_per_struct]
        # break

dsc_n2p2 = {elem: np.array(dsc_n2p2[elem]) for elem in dsc_n2p2}
for elem in dsc_n2p2:
    print(elem, dsc_n2p2[elem].shape)
    # print(dsc_tip[elem][0])

In [ ]:
for elem in dsc_tip:
    print(elem)
    assert np.allclose(dsc_n2p2[elem], dsc_tip[elem])

In [ ]:
for elem in dsc_tip:
    plt.hist((dsc_n2p2[elem] - dsc_tip[elem]).flatten());
    plt.xlabel('error'); plt.ylabel('count'); plt.title(elem);
    plt.show()